In [1]:
import numpy as np
np.set_printoptions(threshold = np.nan, linewidth = 115)
import pickle
import keras
from keras import models, optimizers, layers, regularizers, metrics, losses
from keras.layers.advanced_activations import LeakyReLU, PReLU, ELU, ReLU, ThresholdedReLU
from keras.layers.core import Dense, Dropout, SpatialDropout2D, Activation
from keras.layers.convolutional import Conv2D, SeparableConv2D
from keras.models import model_from_json, Sequential

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config = config)

IMG_SIZE = 256

# Save Comparison model
def save_model(model_name, hist_str, model_str):

    pickle.dump(model_name.history, open('Training Histories/'+ hist_str + '.p', 'wb'))
    
    print("Saved " + hist_str + " to Training Histories folder")
    
    # serialize model to JSON
    model_name = model.to_json()
    with open("CNN Models/" + model_str + ".json", "w") as json_file:
        json_file.write(model_name)

    # serialize weights to HDF5
    model.save_weights("CNN Models/" + model_str + ".h5")
    print("Saved " + model_str + " and weights to CNN Models folder")
    
# Load model architecture and weights NOTE: must compile again
def load_model():
    model_str = str(input("Name of model to load: "))

    # load json and create model
    json_file = open('CNN Models/' + model_str + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("CNN Models/" + model_str + ".h5")
    print("Loaded " + model_str + " and weights from CNN Models folder")
    
    return loaded_model
    
# Load history object
def load_history():
    hist_str = str(input("Name of history to load: "))

    loaded_history = pickle.load(open('Training Histories/' + hist_str + '.p', 'rb'))
    
    print("Loaded " + hist_str + " from Training Histories folder")
    
    return loaded_history

Infiltration_class_weight = [{0: 1, 1: 12.9379183014078}]

Using TensorFlow backend.


In [2]:
# Load with pickle instead of processing images again
training_img_1 = pickle.load(open('1vAll_img_res_Infiltration_1st_half.p', 'rb'))
training_img_2 = pickle.load(open('1vAll_img_res_Infiltration_2nd_half.p', 'rb'))

training_img = np.append(training_img_1, training_img_2, axis = 0)

test_img = pickle.load(open('1vAll_test_img.p', 'rb'))

labels_1 = pickle.load(open('1vAll_labels_res_Infiltration_1st_half.p', 'rb'))
labels_2 = pickle.load(open('1vAll_labels_res_Infiltration_2nd_half.p', 'rb'))

training_labels = np.append(labels_1, labels_2, axis = 0)

test_labels = pickle.load(open('1vAll_test_labels.p', 'rb'))    

In [3]:
Infiltration_labels = training_labels[:, 8]
print('Labels shape: ', Infiltration_labels.shape)
print('Length of test_labels: ', len(test_labels))
print('No. of Infiltration Diagnoses: ', sum(Infiltration_labels))

Labels shape:  (46184,)
Length of test_labels:  3490
No. of Infiltration Diagnoses:  18149


In [11]:
# Metric Analysis
def _1vAll_accuracy(y_test, pred):
    
    pred = np.squeeze(pred, axis = -1)
    pred = np.round_(pred)
    pred = pred.astype(dtype = 'uint8')
    
    ft = pred == y_test
    
    accuracy = sum(ft)/len(ft)
        
    print('\t Complete Label Accuracy: ', (accuracy * 100), '%')
    
    print('Sum of Fully Correct Predictions: ', sum(ft))
    print('\t\t    Total Labels: ', len(ft))
    
    return accuracy

In [8]:
### Custom Loss function
from keras import backend as K

In [9]:
def LSEP(true_label, pred_label):
    return K.log(1 + K.sum(K.exp(true_label - pred_label)))

### One Vs. All

In [5]:
# Infiltration One Vs. All

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Last model: Infiltration_1vAll_30e
#model = load_model()
#model.summary()

model.compile(optimizer = optimizers.RMSprop(lr = 1e-3), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

model_obj = model.fit(training_img, Infiltration_labels, 
                      epochs = 30, initial_epoch = 0, 
                      validation_split = (1 / 8), 
                      batch_size = 128, verbose = 1 )
                     #, class_weight = Infiltration_class_weight)

Predictions = model.predict(test_img)

Accuracy = _1vAll_accuracy(test_labels, Predictions)

history_str = 'Infiltration_1vAll_30e_history'
model_str   = 'Infiltration_1vAll_30e'
    
save_model(model_obj, history_str, model_str)

Train on 40411 samples, validate on 5773 samples
Epoch 1/30
40411/40411 [==============================] - 76s 2ms/step - loss: 6.3726 - acc: 0.6034 - val_loss: 5.9497 - val_acc: 0.6309
Epoch 2/30
40411/40411 [==============================] - 72s 2ms/step - loss: 6.3888 - acc: 0.6036 - val_loss: 5.9497 - val_acc: 0.6309
Epoch 3/30
40411/40411 [==============================] - 72s 2ms/step - loss: 6.3888 - acc: 0.6036 - val_loss: 5.9497 - val_acc: 0.6309
Epoch 4/30
40411/40411 [==============================] - 73s 2ms/step - loss: 6.3888 - acc: 0.6036 - val_loss: 5.9497 - val_acc: 0.6309
Epoch 5/30
40411/40411 [==============================] - 73s 2ms/step - loss: 6.3888 - acc: 0.6036 - val_loss: 5.9497 - val_acc: 0.6309
Epoch 6/30
40411/40411 [==============================] - 73s 2ms/step - loss: 6.3888 - acc: 0.6036 - val_loss: 5.9497 - val_acc: 0.6309
Epoch 7/30
40411/40411 [==============================] - 73s 2ms/step - loss: 6.3888 - acc: 0.6036 - val_loss: 5.9497 - val_acc:

In [6]:
# One Vs. All - Inception Model

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='elu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(SpatialDropout2D(0.375))
model.add(layers.Conv2D(64, (3, 3), activation='elu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SeparableConv2D(16, (3, 3), activation='elu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(Dropout(0.375))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='elu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Last model: 1vAll_Inception_30e
#model = load_model()
#model.summary()

model.compile(optimizer = optimizers.RMSprop(lr = 1e-3), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

model_obj = model.fit(training_img, Infiltration_labels, 
                      epochs = 30, initial_epoch = 0, 
                      validation_split = (1 / 8), 
                      batch_size = 128, verbose = 1 )
                     #, class_weight = Infiltration_class_weight)

Predictions = model.predict(test_img)

Accuracy = _1vAll_accuracy(test_labels, Predictions)

history_str = '1vAll_Inception_30e_history'
model_str   = '1vAll_Inception_30e'
    
save_model(model_obj, history_str, model_str)

Train on 40411 samples, validate on 5773 samples
Epoch 1/30
40411/40411 [==============================] - 94s 2ms/step - loss: 1.1271 - acc: 0.5900 - val_loss: 0.6600 - val_acc: 0.6309
Epoch 2/30
40411/40411 [==============================] - 103s 3ms/step - loss: 0.6759 - acc: 0.6030 - val_loss: 0.6872 - val_acc: 0.5711
Epoch 3/30
40411/40411 [==============================] - 106s 3ms/step - loss: 0.6873 - acc: 0.6049 - val_loss: 0.6718 - val_acc: 0.6182
Epoch 4/30
40411/40411 [==============================] - 102s 3ms/step - loss: 0.6691 - acc: 0.6090 - val_loss: 0.6455 - val_acc: 0.6385
Epoch 5/30
40411/40411 [==============================] - 94s 2ms/step - loss: 0.6656 - acc: 0.6111 - val_loss: 0.6587 - val_acc: 0.6212
Epoch 6/30
40411/40411 [==============================] - 100s 2ms/step - loss: 0.6590 - acc: 0.6137 - val_loss: 0.6580 - val_acc: 0.6246
Epoch 7/30
40411/40411 [==============================] - 100s 2ms/step - loss: 0.6555 - acc: 0.6183 - val_loss: 0.6683 - val

In [7]:
# One Vs. All - Inception Model
# v2: Using Class Weights

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='elu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(SpatialDropout2D(0.375))
model.add(layers.Conv2D(64, (3, 3), activation='elu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SeparableConv2D(16, (3, 3), activation='elu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(Dropout(0.375))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='elu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Last model: 1vAll_Inception_v2_30e
#model = load_model()
#model.summary()

model.compile(optimizer = optimizers.RMSprop(lr = 1e-3), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

model_obj = model.fit(training_img, Infiltration_labels, 
                      epochs = 30, initial_epoch = 0, 
                      validation_split = (1 / 8), 
                      batch_size = 128, verbose = 1 , 
                      class_weight = Infiltration_class_weight)

Predictions = model.predict(test_img)

Accuracy = _1vAll_accuracy(test_labels, Predictions)

history_str = '1vAll_Inception_v2_30e_history'
model_str   = '1vAll_Inception_v2_30e'
    
save_model(model_obj, history_str, model_str)

Train on 40411 samples, validate on 5773 samples
Epoch 1/30
40411/40411 [==============================] - 92s 2ms/step - loss: 9.7375 - acc: 0.3974 - val_loss: 10.0575 - val_acc: 0.3691
Epoch 2/30
40411/40411 [==============================] - 100s 2ms/step - loss: 9.6232 - acc: 0.3964 - val_loss: 10.0575 - val_acc: 0.3691
Epoch 3/30
40411/40411 [==============================] - 99s 2ms/step - loss: 9.6231 - acc: 0.3964 - val_loss: 10.0575 - val_acc: 0.3691
Epoch 4/30
40411/40411 [==============================] - 103s 3ms/step - loss: 9.6228 - acc: 0.3964 - val_loss: 10.0575 - val_acc: 0.3691
Epoch 5/30
40411/40411 [==============================] - 110s 3ms/step - loss: 9.6232 - acc: 0.3964 - val_loss: 10.0575 - val_acc: 0.3691
Epoch 6/30
40411/40411 [==============================] - 97s 2ms/step - loss: 9.6228 - acc: 0.3964 - val_loss: 10.0575 - val_acc: 0.3691
Epoch 7/30
40411/40411 [==============================] - 104s 3ms/step - loss: 9.6232 - acc: 0.3964 - val_loss: 10.0575

In [10]:
# Infiltration One Vs. All - Custom Loss Functions

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Last model: Infiltration_1vAll_20e
#model = load_model()
#model.summary()

model.compile(optimizer = optimizers.RMSprop(lr = 1e-3), 
              loss = LSEP, 
              metrics = ['accuracy'])

model_obj = model.fit(training_img, Infiltration_labels, 
                      epochs = 20, initial_epoch = 0, 
                      validation_split = (1 / 8), 
                      batch_size = 128, verbose = 1 )
                     #, class_weight = Infiltration_class_weight)

Predictions = model.predict(test_img)

Accuracy = _1vAll_accuracy(test_labels, Predictions)

history_str = '1vAll_Custom_Loss_20e_history'
model_str   = '1vAll_Custom_Loss_20e'
    
save_model(model_obj, history_str, model_str)

Train on 40411 samples, validate on 5773 samples
Epoch 1/20
40411/40411 [==============================] - 73s 2ms/step - loss: 4.3825 - acc: 0.3964 - val_loss: 4.2673 - val_acc: 0.3691
Epoch 2/20
40411/40411 [==============================] - 78s 2ms/step - loss: 4.3823 - acc: 0.3964 - val_loss: 4.2673 - val_acc: 0.3691
Epoch 3/20
40411/40411 [==============================] - 79s 2ms/step - loss: 4.3824 - acc: 0.3964 - val_loss: 4.2673 - val_acc: 0.3691
Epoch 4/20
40411/40411 [==============================] - 82s 2ms/step - loss: 4.3824 - acc: 0.3964 - val_loss: 4.2673 - val_acc: 0.3691
Epoch 5/20
40411/40411 [==============================] - 81s 2ms/step - loss: 4.3824 - acc: 0.3964 - val_loss: 4.2673 - val_acc: 0.3691
Epoch 6/20
40411/40411 [==============================] - 78s 2ms/step - loss: 4.3824 - acc: 0.3964 - val_loss: 4.2673 - val_acc: 0.3691
Epoch 7/20
40411/40411 [==============================] - 81s 2ms/step - loss: 4.3823 - acc: 0.3964 - val_loss: 4.2673 - val_acc: